In [6]:
import pandas as pd

In [7]:
!head -n 1000 violations.csv > small-violations.csv

head: violations.csv: No such file or directory


## 1) I want to make sure my Plate ID is a string. Can't lose the leading zeroes!

In [8]:
col_types = {'Plate ID': 'str'}
df = pd.read_csv("small-violations.csv", dtype=col_types)

EmptyDataError: No columns to parse from file

In [9]:
df.head()

NameError: name 'df' is not defined

## 2) I don't think anyone's car was built in 0AD. Discard the '0's as NaN.

In [ ]:
col_types = {'Plate ID': 'str'}
df = pd.read_csv("small-violations.csv", dtype=col_types, na_values={'Vehicle Year': '0', 'Date First Observed': '0'})
df.head()

## 3) I want the dates to be dates! Read the read_csv documentation to find out how to make pandas automatically parse dates.

In [ ]:
import dateutil
def date_to_date(date):
        date = str(date)
        clean_date = dateutil.parser.parse(date)
        return clean_date

In [ ]:
df.columns

In [ ]:
df['New Issue Date']= df['Issue Date'].apply(date_to_date)

In [ ]:
import datetime
def convert_to_time(time):
    try:
        str_time = str(time)
        return datetime.datetime.strptime(str_time, "%Y%m%d")
    except:
        return None

In [ ]:
new_df = df[df['Vehicle Expiration Date'] != 0]
new_df.head()

In [ ]:
new_df['New Vehicle Expiration Date']= new_df['Vehicle Expiration Date'].apply(convert_to_time)

In [ ]:
new_df.head()

## 4) "Date first observed" is a pretty weird column, but it seems like it has a date hiding inside. Using a function with .apply, transform the string (e.g. "20140324") into a Python date. Make the 0's show up as NaN

In [ ]:
new_df.columns

In [ ]:
new_df['Date First Observed'].dtypes

In [ ]:
new_df['Date First Observed'].tail()

In [ ]:
import dateutil

In [ ]:
new_df['Date First Observed']

In [ ]:
new_df['Violation Time'].head()

In [ ]:
def int_to_date(integer):
    if not pd.isnull(integer):
        date = str(int(integer))
        clean_date = dateutil.parser.parse(date)
        return clean_date.strftime("%Y-%-m-%d")

In [ ]:
new_df['Date First Observed'].apply(int_to_date)

## 5) "Violation time" is... not a time. Make it a time.

In [ ]:
def violation_time_to_time(time):
    try:
        hour = time[0:2]
        minutes = time[2:4]
        am_pm= time[4]
        complete_time= hour + ":" + minutes + " " + am_pm + 'm'
        new_violation_time = dateutil.parser.parse(complete_time)
        return new_violation_time.strftime("%H:%M%p")
    except:
        return None   

In [ ]:
new_df['Violation Time'].apply(violation_time_to_time)

## 6) There sure are a lot of colors of cars, too bad so many of them are the same. Make "BLK" and "BLACK", "WT" and "WHITE", and any other combinations that you notice.

In [ ]:
new_df['Vehicle Color'].value_counts()

In [ ]:
def color_to_name(color):
    if (color == 'BLACK') or (color == 'BLK') or (color == 'BK'):
        return 'BLACK'
    elif (color == 'WHITE') or (color == 'WHT') or (color == 'WH') or (color == 'W'):
        return 'WHITE'

new_df['Vehicle Color'].apply(color_to_name)

## 7) Join the data with the Parking Violations Code dataset from the NYC Open Data site

In [ ]:
parking_violations_df = pd.read_csv("DOF_Parking_Violation_Codes.csv", encoding="mac_roman", error_bad_lines=False)
parking_violations_df.head()

In [ ]:
parking_violations_df['CODE'].describe()

In [ ]:
new_df['Violation Code'].describe()

In [ ]:
def convert_to_str(n):
    return str(n)

In [ ]:
parking_violations_df['Code'] = parking_violations_df['CODE'].apply(convert_to_str)

In [ ]:
new_df['Violation code'] = new_df['Violation Code'].apply(convert_to_str)

In [ ]:
parking_violations_df.head()

In [ ]:
new_parking_violations_df = parking_violations_df.rename(columns={'Manhattan¬† 96th St. & below': 'Manhattan 96th & below', 'All Other Areas': 'All other areas'})
new_parking_violations_df.head()

In [ ]:
new_df.head()

In [ ]:
new_violations_df = pd.merge(new_df, new_parking_violations_df, left_on='Violation code', right_on='Code')
new_violations_df.head()

## 8) How much money did NYC make off of parking violations?

In [ ]:
new_violations_df['Manhattan 96th & below'].describe()

In [ ]:
new_violations_df['All other areas'].describe()

In [ ]:
new_violations_df['Manhattan 96th & below'].apply(convert_to_str).head()

In [ ]:
new_violations_df['All other areas'].apply(convert_to_str).head()

In [ ]:
new_violations_df = new_violations_df[new_violations_df['Manhattan 96th & below'] != 'vary']
new_violations_df.head()

In [ ]:
import re
def strip_and_convert_to_int(string):
    match = re.findall(r"^\$?\d*", string)
    if match:
        new_string = string.replace("$", "").split()
        new_int = int(new_string[0])
        return new_int
    else:
        return None
    

In [ ]:
new_violations_df['Manhattan 96th and below'] = new_violations_df['Manhattan 96th & below'].apply(strip_and_convert_to_int)

In [ ]:
new_violations_df.head()

In [ ]:
new_violations_df['All Other Areas'] = new_violations_df['All other areas'].apply(strip_and_convert_to_int)
new_violations_df.tail()

In [ ]:
new_violations_df['All Other Areas'].value_counts().head()

In [ ]:
violations_per_code_manhattan = new_violations_df.groupby('Violation code')['All Other Areas'].sum()
violations_per_code_manhattan.sum()

In [ ]:
violations_per_code_rest = new_violations_df.groupby('Violation code')['Manhattan 96th and below'].sum()
violations_per_code_rest.sum()

In [ ]:
total_violations_income = violations_per_code_rest.sum() + violations_per_code_manhattan.sum()

In [ ]:
total_violations_income

## 9) What's the most lucrative kind of parking violation? The most frequent?

In [ ]:
violations_per_code_manhattan.sort_values(ascending=False)

In [ ]:
violations_per_code_rest.sort_values(ascending=False)

In [ ]:
new_violations_df['Violation code'].value_counts()

## 10) New Jersey has bad drivers, but does it have bad parkers, too? How much money does NYC make off of all non-New York vehicles?

In [ ]:
non_ny_vehicles_df = new_violations_df[new_violations_df['Registration State'] != 'NY']
non_ny_vehicles_df.head()

In [ ]:
violations_per_code_non_ny_other = non_ny_vehicles_df.groupby('Violation code')['All Other Areas'].sum()
violations_per_code_non_ny_other.sum()

In [ ]:
violations_per_code_non_ny_manhattan= non_ny_vehicles_df.groupby('Violation code')['Manhattan 96th and below'].sum()
violations_per_code_non_ny_manhattan.sum()

In [ ]:
all_violations_non_ny = violations_per_code_non_ny_other.sum()+violations_per_code_non_ny_manhattan.sum()
all_violations_non_ny

## 11) Make a chart of the top few

In [ ]:
%matplotlib inline

In [ ]:
violations_per_code_non_ny_other.sort_values(ascending=False).plot(kind='bar', x='Violation code')

In [ ]:
violations_per_code_non_ny_manhattan.sort_values(ascending=False).plot(kind='bar', x='Violation code')

## 12) What time of day do people usually get their tickets? You can break the day up into several blocks - for example12am-6am, 6am-12pm, 12pm-6pm, 6pm-12am.

## 13) What's the average ticket cost in NYC?

In [ ]:
average_ticket_cost = total_violations_income / new_violations_df['Violation code'].value_counts().sum()
average_ticket_cost

## 14) Make a graph of the number of tickets per day.

In [ ]:
new_violations_df['Issue Date'].value_counts().head(10).plot(kind='barh')

## 15) Make a graph of the amount of revenue collected per day.

In [ ]:
revenue_per_day = total_violations_income / new_violations_df['New Issue Date'].value_counts()
revenue_per_day.sort_values(ascending=False).head(20).plot(kind='bar')

## 16) Manually construct a dataframe out of https://dmv.ny.gov/statistic/2015licinforce-web.pdf (only NYC boroughts - bronx, queens, manhattan, staten island, brooklyn), having columns for borough name, abbreviation, and number of licensed drivers.

In [ ]:
nyc_licenses_per_borough = pd.read_excel("NYC.xlsx")
nyc_licenses_per_borough

## 17) What's the parking-ticket-$-per-licensed-driver in each borough of NYC? Do this with pandas and the dataframe you just made, not with your head!

In [ ]:
new_violations_df.columns

In [ ]:
new_violations_df['Violation County'].value_counts()

In [ ]:
violations_bronx = new_violations_df[new_violations_df['Violation County'] == 'BX']
licenses_bronx = nyc_licenses_per_borough['Total'][nyc_licenses_per_borough['Abbreviation'] == 'BX']
tickets_bronx = violations_bronx.groupby('Violation code')['All Other Areas'].sum()
ticket_per_driver_bronx = licenses_bronx / tickets_bronx.sum()
ticket_per_driver_bronx

In [ ]:
violations_queens = new_violations_df[new_violations_df['Violation County'] == 'Q']
licenses_queens = nyc_licenses_per_borough['Total'][nyc_licenses_per_borough['Abbreviation'] == 'Q']
tickets_queens = violations_queens.groupby('Violation code')['All Other Areas'].sum()
ticket_per_driver_queens = licenses_queens / tickets_queens.sum()
ticket_per_driver_queens

In [ ]:
violations_ny = new_violations_df[new_violations_df['Violation County'] == 'NY']
licenses_ny = nyc_licenses_per_borough['Total'][nyc_licenses_per_borough['Abbreviation'] == 'NY']
tickets_ny = violations_ny.groupby('Violation code')['Manhattan 96th and below'].sum()
ticket_per_driver_ny = licenses_ny / tickets_ny.sum()
ticket_per_driver_ny

In [ ]:
violations_richmond = new_violations_df[new_violations_df['Violation County'] == 'R']
licenses_richmond = nyc_licenses_per_borough['Total'][nyc_licenses_per_borough['Abbreviation'] == 'R']
tickets_richmond = violations_richmond.groupby('Violation code')['All Other Areas'].sum()
ticket_per_driver_richmond = licenses_richmond / tickets_richmond.sum()
ticket_per_driver_richmond

In [ ]:
violations_kings = new_violations_df[new_violations_df['Violation County'] == 'K']
licenses_kings = nyc_licenses_per_borough['Total'][nyc_licenses_per_borough['Abbreviation'] == 'K']
tickets_kings = violations_kings.groupby('Violation code')['All Other Areas'].sum()
ticket_per_driver_kings = licenses_kings / tickets_kings.sum()
ticket_per_driver_kings